1. get indicator dataframe <-> indicator key from website
2. make a function to check the country name
3. update dataframe


We import pandas and json

In [2]:
import pandas as pd
import json
import os.path

We read the df previously saved in a file in order to get back the df in the workspace, or we load the excel file for the first execution

In [3]:
if os.path.isfile('savedDF.pickle') or os.path.isfile('Data-sheet-STUDENTS.xlsx'):
    if os.path.isfile('savedDF.pickle'):
        df = pd.read_pickle("savedDF.pickle")
        print("DF saved loaded")
    else:
        df = pd.read_excel('Data-sheet-STUDENTS.xlsx', header=2, index_col=0)
        print("DF from the empty execl loaded")
else:
    print("Error: no file to load the df exist in the current folder")

DF saved loaded


We get the data from belt and road portal by downloading previously a json file. We read this file under.

In [4]:
with open('data-query-20170525.json', 'r') as fp:
    data = json.load(fp)

In [ ]:
data

In [9]:
data.keys()

dict_keys(['country_macro', 'china_macro', 'invest_trade', 'state_economy'])

In [10]:
data['country_macro']['term'].keys()

dict_keys(['year', 'area', 'item'])

In [11]:
dicoYear = {
    '2015': '65d2ea03425887a717c435081cfc5dbb', 
    '2014': 'cee8d6b7ce52554fd70354e37bbf44a2', 
    '2013': '8038da89e49ac5eabb489cfc6cea9fc1'
}
dicoIndicator = {
    'BRI GDP': '9b688b5fbb441714ba73f4379e7b135d', 
    'BRI GDP p.c.': '6d602224213550f3ee07477f1654f326', 
    'BRI GDP growth rate': '5c4591fe4746333f6e7b66b13ad980a2',
    'BRI Y.o.Y. GDP growth rate p.c.': '07f451a8c9b2006fb937d1eb268ac60b',
    'BRI Agriculture value added per sector ': 'c1329b4c01d4c08a68b8a280fb5a90ec',
    'BRI Industry value added per sector ': 'f00273fa66d45147b2dd4dadd6275d32',
    'BRI Services value added per sector ': '41fedd97a26d4782e531b5b4c67622b7',
    'BRI Internet penetration rate (%)': '7b8d0a597b57bab793f428193eb899fe',
    'BRI Population density (/km2)': '2a048dbe6ff2ee51d07a6314752c87d7',
    'BRI Percentage of Urban Population in Total Population': '85bf6dab128029be54a3b1e643f0aa71'
}

#'Exp./Imp. volume USD': '', this one is not in the json

In [12]:
def getCountryCode(countryName):
    for k,v in data['country_macro']['term']['area'].items():
        code = k
        if v['full'] == countryName or v['short'] == countryName:
            return code
    return None

In [ ]:
for nameYear, codeYear in dicoYear.items():
    for nameColumn, codeIndicator in dicoIndicator.items():
        for indexCountryDf, countryDf in df['Country (according to CN Gov):'].items():
            #if we don't have the value in the DF we try to find it in the json.
            if pd.isna(df.at[indexCountryDf,nameColumn]):
                codeCountry = getCountryCode(countryDf)
                if codeCountry is not None:
                    try:
                        value = float(data['country_macro']['data'][codeYear][codeCountry][codeIndicator])
                        df[nameColumn].loc[indexCountryDf] = value
                    except:
                        print('Error - no value found', nameYear, nameColumn, countryDf)

        

We save the DF

In [16]:
df.to_pickle("savedDF.pickle")